In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

## Login

In [2]:
ctg_url = 'https://cleaningtheglass.com'

In [3]:
ctg_login_url = 'https://cleaningtheglass.memberful.com/auth/sign_in'

In [4]:
def login(driver, cred, ctg_login_url=ctg_login_url):
    driver.get(ctg_login_url)
    for button in driver.find_elements_by_tag_name('button'):
        if button.text == 'sign in using password':
            button.click()
            break
    # Get username & password
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')
    # Login
    userfield = driver.find_element_by_id('session_email')
    userfield.send_keys(user_password[0])
    passwordfield = driver.find_element_by_id('session_password')
    passwordfield.send_keys(user_password[1])
    for button in driver.find_elements_by_class_name('button'):
        if button.get_attribute('value') == 'Sign in':
            button.click()
            break

In [5]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/3545715976.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/local/bin/chromedriver')


In [6]:
cred = 'ctg_cred.txt'
login(driver, cred)

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/1453808485.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for button in driver.find_elements_by_tag_name('button'):
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/1453808485.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  userfield = driver.find_element_by_id('session_email')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/1453808485.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passwordfield = driver.find_element_by_id('session_password')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/1453808485.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for button in driver.find_elements_by_class_name('button'):


Get team dict

In [8]:
driver.get(f'{ctg_url}/stats/teams')
team_container = driver.find_element_by_id('team_container')
team_dict = {a.get_attribute('href').split('/')[-1]:
             a.find_element_by_tag_name('img').get_attribute('alt')
             for a in team_container.find_elements_by_tag_name('a')}
team_dict

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_31433/219544807.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  team_container = driver.find_element_by_id('team_container')
/Users/xulian/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/Users/xulian/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


{'1': 'Atlanta Hawks',
 '2': 'Boston Celtics',
 '3': 'Brooklyn Nets',
 '4': 'Charlotte Hornets',
 '5': 'Chicago Bulls',
 '6': 'Cleveland Cavaliers',
 '7': 'Dallas Mavericks',
 '8': 'Denver Nuggets',
 '9': 'Detroit Pistons',
 '10': 'Golden State Warriors',
 '11': 'Houston Rockets',
 '12': 'Indiana Pacers',
 '13': 'Los Angeles Clippers',
 '14': 'Los Angeles Lakers',
 '15': 'Memphis Grizzlies',
 '16': 'Miami Heat',
 '17': 'Milwaukee Bucks',
 '18': 'Minnesota Timberwolves',
 '19': 'New Orleans Pelicans',
 '20': 'New York Knicks',
 '21': 'Oklahoma City Thunder',
 '22': 'Orlando Magic',
 '23': 'Philadelphia 76ers',
 '24': 'Phoenix Suns',
 '25': 'Portland Trail Blazers',
 '26': 'Sacramento Kings',
 '27': 'San Antonio Spurs',
 '28': 'Toronto Raptors',
 '29': 'Utah Jazz',
 '30': 'Washington Wizards'}

## Basic Function

In [9]:
def fetch(driver):
    """
    Use BeautifulSoup to get the text of a web page.
    """
    d = driver.page_source
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", d), "html.parser")
    return soup

## Download Lineup Data

In [10]:
def scrape_team_lineup_table(team_id, season, driver,
                             team_dict=team_dict):
    driver.get(f'{ctg_url}/stats/team/{team_id}/lineups?season={season}')
    time.sleep(3)
    soup = fetch(driver)
    table = soup.find_all('table', {'id': 'lineups_four_factors'})[0]
    # Build dataframe
    cols = ['PG', 'SG', 'SF', 'PF', 'C', 'Poss', 'Diff_Rank', 'Diff']
    for side in ['off', 'def']:
        for factor in ['Pts/Poss', 'eFG%', 'TOV%', 'OREB%', 'FTr']:
            cols = cols + [f'{side}_{factor}_rank', f'{side}_{factor}']
        if side == 'off':
            cols = cols + ['']
    data = [[td['data-full-name'] for td in row.find_all('td')[:5]] +
            [td.text for td in row.find_all('td')[5:]]
            for row in table.find_all('tbody')[1].find_all('tr')]
    df = pd.DataFrame(data, columns=cols).drop('', axis=1)
    df['season'] = season
    df['team'] = team_dict[team_id]
    return df

In [11]:
print('Start Scraping...')
team_dfs = {}
for team_id in team_dict.keys():
    team = team_dict[team_id]
    print(team)
    team_dfs[team] = pd.concat([scrape_team_lineup_table(team_id, season, driver)
                                for season in range(2016, 2022)])
print('Done.')

Start Scraping...
Atlanta Hawks
Boston Celtics
Brooklyn Nets
Charlotte Hornets
Chicago Bulls
Cleveland Cavaliers
Dallas Mavericks
Denver Nuggets
Detroit Pistons
Golden State Warriors
Houston Rockets
Indiana Pacers
Los Angeles Clippers
Los Angeles Lakers
Memphis Grizzlies
Miami Heat
Milwaukee Bucks
Minnesota Timberwolves
New Orleans Pelicans
New York Knicks
Oklahoma City Thunder
Orlando Magic
Philadelphia 76ers
Phoenix Suns
Portland Trail Blazers
Sacramento Kings
San Antonio Spurs
Toronto Raptors
Utah Jazz
Washington Wizards
Done.


In [12]:
driver.close()

In [13]:
df = pd.concat([team_dfs[team] for team in team_dict.values()])\
       .reset_index().drop('index', axis=1)

In [14]:
df.to_csv('ctg_nba_lineup_stats.csv', index=False)